# Prediction of Singlet Triplet Gaps by Deep Learning 
The following code shows how our models can be restored from their final checkpoints to do ST gap and HL gap predictions on potential photosensitizer structures with their SMILES taken as inputs. 

In [1]:
import numpy as np
from rdkit import Chem
import tensorflow as tf 
import deepchem as dc 
from models.graphConvModel import GraphConvModel

import warnings
warnings.filterwarnings('ignore')
import logging
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

tf.random.set_seed(42)
np.random.seed(42)

In [2]:
list_names = ["DAn_694", "DAn_4493", "DAn_1003", "DADx_21"]
list_SMILES = [
    "N#CC(C#N)=c1c2ccccc2c(=C(C#N)C#N)c2cc([N]3c4ccccc4Sc4ccccc43)ccc12", 
    "O=C1c2ccccc2C(=O)c2cc(-c3c4ccccc4c(N4c5ccccc5Oc5ccccc54)c4nsnc34)ccc21",
    "O=C1c2ccccc2C(=O)c2cc(-n3c4ccccc4c(=O)c4ccccc43)ccc21",
    "N#CC(C(C#N)=N1)=NC2=C1C(C=CC(C3=CC=C(N(C4=CC=CC=C4)C5=CC=CC=C5)C=C3)=C6)=C6C7=C2C=CC(C8=CC=C(N(C9=CC=CC=C9)C%10=CC=CC=C%10)C=C8)=C7"
]

# feature SMILES
graph_featurizer = dc.feat.graph_features.ConvMolFeaturizer()
mols = [Chem.MolFromSmiles(smi) for smi in list_SMILES]
graphs = graph_featurizer.featurize(mols)
DA_data = dc.data.NumpyDataset(graphs[:-1])
DAD_data = dc.data.NumpyDataset([graphs[-1]])

In [3]:
# restore model trained on donor-acceptor structures (DA) 
model_dir = "DA_Final_Model" # NOTE: Be sure to have the model files saved in the same directory with this name 
DA_model = GraphConvModel(n_tasks = 2,
                       graph_conv_layers = [295, 295, 295, 295, 295, 295],
                       dense_layers = [382, 382, 382, 382],
                       dropout = 0.00874,
                       learning_rate = 0.0001,
                       batch_size = 10,
                       model_dir = model_dir)
DA_model.restore(DA_model.get_checkpoints()[-1])

# predict with the mol data 
DA_pred = DA_model.predict(DA_data)
for i, pred in enumerate(DA_pred):
    print(f"{list_names[i]} ST gap: {pred[0]:.4f} eV | HL gap: {pred[1]*27.2114:.4f} eV")


# restore model trained on donor-acceptor-donor structures (DAD) 
model_dir = "DAD_Final_Model" # NOTE: Be sure to have the model files saved in the same directory with this name 
DAD_model = GraphConvModel(n_tasks = 2,
                       graph_conv_layers = [512, 512, 512, 512], 
                       dense_layers = [128, 128, 128],
                       dropout = 0.01,
                       learning_rate = 0.001,
                       batch_size = 10,
                       model_dir = model_dir)
DAD_model.restore(DAD_model.get_checkpoints()[-1])

# predict with the mol data 
DAD_pred = DAD_model.predict(DAD_data)
print(f"{list_names[-1]} ST gap: {DAD_pred[0][0]:.4f} eV | HL gap: {DAD_pred[0][1]*27.2114:.4f} eV")


DAn_694 ST gap: 0.1652 eV | HL gap: 1.8458 eV
DAn_4493 ST gap: 0.0141 eV | HL gap: 1.5665 eV
DAn_1003 ST gap: 0.1183 eV | HL gap: 2.8229 eV
DADx_21 ST gap: 0.1989 eV | HL gap: 2.5847 eV
